### Preparing Data ###

In [1]:
import pandas as pd
from scipy.spatial import cKDTree
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/Хакатон/dataset2015_2019.csv')

In [7]:
data.head(2)

,Unnamed: 0,centroid,x,y,RotationLength,RotationCount,CODE_CULTU_2015,CODE_CULTU_2016,CODE_CULTU_2017,CODE_CULTU_2018,CODE_CULTU_2019,CODE_GROUP_2015,CODE_GROUP_2016,CODE_GROUP_2017,CODE_GROUP_2018,CODE_GROUP_2019,cultuCode,groupCode,hzs,kg_id,ff_id,1 Nearest Neigbour Point,2 Nearest Neigbour Point,3 Nearest Neigbour Point,4 Nearest Neigbour Point
0,0,Point (907753.82834152 6554634.46488151),907753.828342,6.554634e+06,1,5,PPH,PPH,PPH,PPH,PPH,18,18,18,18,18,aaaaa,aaaaa,7b,Dfb,ff12,Point (907748.69100314 6554212.06490037),Point (907410.62452802 6554909.79074054),Point (908089.34400838 6554953.05455597),Point (907563.65755459 6554039.35762193)
1,1,Point (906932.34642625 6550537.14396784),906932.346426,6.550537e+06,1,5,PPH,PPH,PPH,PPH,PPH,18,18,18,18,18,aaaaa,aaaaa,7b,Dfb,ff12,Point (906980.7207283 6550304.97315954),Point (906736.32315476 6550335.52813463),Point (906595.72670333 6550598.70621165),Point (907032.95563938 6550121.6369531)


In [6]:
!pip install catboost

     |████████████████████████████████| 65.7MB 88kB/s 


In [8]:
import catboost as cb
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import GridSearchCV

In [106]:
data.head(2)

,Unnamed: 0,centroid,x,y,RotationLength,RotationCount,CODE_CULTU_2015,CODE_CULTU_2016,CODE_CULTU_2017,CODE_CULTU_2018,CODE_CULTU_2019,CODE_GROUP_2015,CODE_GROUP_2016,CODE_GROUP_2017,CODE_GROUP_2018,CODE_GROUP_2019,cultuCode,groupCode,hzs,kg_id,ff_id,1 Nearest Neigbour Point,2 Nearest Neigbour Point,3 Nearest Neigbour Point,4 Nearest Neigbour Point
0,0,Point (907753.82834152 6554634.46488151),907753.828342,6.554634e+06,1,5,PPH,PPH,PPH,PPH,PPH,18,18,18,18,18,aaaaa,aaaaa,7b,Dfb,ff12,Point (907748.69100314 6554212.06490037),Point (907410.62452802 6554909.79074054),Point (908089.34400838 6554953.05455597),Point (907563.65755459 6554039.35762193)
1,1,Point (906932.34642625 6550537.14396784),906932.346426,6.550537e+06,1,5,PPH,PPH,PPH,PPH,PPH,18,18,18,18,18,aaaaa,aaaaa,7b,Dfb,ff12,Point (906980.7207283 6550304.97315954),Point (906736.32315476 6550335.52813463),Point (906595.72670333 6550598.70621165),Point (907032.95563938 6550121.6369531)


In [9]:
def predict_next_culture_4_years_list(l0, l1, l2, l3):
  if l0 == l1 != l2 and l2 == l3:
     return 'Unknown'
  if l0 == l1 == l2 == l3:
     return l0
  #if l0 == l3:
  #   return l1
  if l0 == l2 and l1 == l3:
     return l0
#  if l0 == l2 and l1 != l2:
#     return l0
  if l0 == l1 != l2 and l2 == l3:
     return l0
  return 'Unknown'

In [10]:
print(predict_next_culture_4_years_list(data.iloc[0]['CODE_CULTU_2015'],data.iloc[0]['CODE_CULTU_2016'],data.iloc[0]['CODE_CULTU_2017'],data.iloc[0]['CODE_CULTU_2018']))

PPH


In [11]:
g = data.groupby(['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'])['CODE_CULTU_2019'].count()

In [12]:
gm = g.add_suffix('').reset_index()

In [14]:
gm_uniq = gm[gm['CODE_CULTU_2019'] == 1]

In [15]:
gm_uniq.head(2)

,CODE_CULTU_2015,CODE_CULTU_2016,CODE_CULTU_2017,CODE_CULTU_2018,CODE_CULTU_2019
0,ACA,FLA,FLA,FLA,1
1,ACA,FLP,FLP,FLP,1


In [16]:
gm_pred = pd.merge(gm_uniq, data,  how='left', left_on=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'], right_on = ['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'])


In [17]:
gm_pred = gm_pred.reindex(columns=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018','CODE_CULTU_2019_y'])

In [18]:
data_pred = pd.merge(data, gm_pred,  how='left', left_on=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'], right_on = ['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'])

In [64]:
data_pred['CODE_CULTU_2019_y'].value_counts()

BTH    11247
PTR     8134
LUZ     4978
MLG     4003
MIS     3902
       ...  
PIS        1
BUR        1
PMV        1
LFP        1
MOL        1
Name: CODE_CULTU_2019_y, Length: 218, dtype: int64

In [65]:
data_part1 = data_pred[data_pred.CODE_CULTU_2019_y.notnull()].copy()

In [66]:
data_part1.head(2)

,Unnamed: 0,centroid,x,y,RotationLength,RotationCount,CODE_CULTU_2015,CODE_CULTU_2016,CODE_CULTU_2017,CODE_CULTU_2018,CODE_CULTU_2019,CODE_GROUP_2015,CODE_GROUP_2016,CODE_GROUP_2017,CODE_GROUP_2018,CODE_GROUP_2019,cultuCode,groupCode,hzs,kg_id,ff_id,1 Nearest Neigbour Point,2 Nearest Neigbour Point,3 Nearest Neigbour Point,4 Nearest Neigbour Point,CODE_CULTU_2019_y
50,50,Point (859900.11098517 6591026.78278985),859900.110985,6.591027e+06,3,1,BTH,MID,MIS,MIS,BTH,1,2,2,2,1,abcca,abbba,8b,Cfa,ff13,Point (859812.49760217 6591185.50103266),Point (859719.0802948 6590975.56193138),Point (860081.26015362 6591098.77150968),Point (860019.18559373 6590736.93947271),BTH
74,74,Point (848022.43324874 6589869.38021483),848022.433249,6.589869e+06,4,1,AVP,SGH,MIS,MIS,ORP,4,4,2,2,3,abccd,aabbc,8b,Cfa,ff13,Point (848060.93345233 6589936.96049742),Point (847937.44700784 6589858.58590304),Point (848069.49079389 6590019.03797348),Point (848120.55312173 6589719.68664306),ORP


In [67]:
data_part2 = data_pred[data_pred.CODE_CULTU_2019_y.isna()].copy()

In [68]:
data_part2.head(2)

,Unnamed: 0,centroid,x,y,RotationLength,RotationCount,CODE_CULTU_2015,CODE_CULTU_2016,CODE_CULTU_2017,CODE_CULTU_2018,CODE_CULTU_2019,CODE_GROUP_2015,CODE_GROUP_2016,CODE_GROUP_2017,CODE_GROUP_2018,CODE_GROUP_2019,cultuCode,groupCode,hzs,kg_id,ff_id,1 Nearest Neigbour Point,2 Nearest Neigbour Point,3 Nearest Neigbour Point,4 Nearest Neigbour Point,CODE_CULTU_2019_y
0,0,Point (907753.82834152 6554634.46488151),907753.828342,6.554634e+06,1,5,PPH,PPH,PPH,PPH,PPH,18,18,18,18,18,aaaaa,aaaaa,7b,Dfb,ff12,Point (907748.69100314 6554212.06490037),Point (907410.62452802 6554909.79074054),Point (908089.34400838 6554953.05455597),Point (907563.65755459 6554039.35762193),NaN
1,1,Point (906932.34642625 6550537.14396784),906932.346426,6.550537e+06,1,5,PPH,PPH,PPH,PPH,PPH,18,18,18,18,18,aaaaa,aaaaa,7b,Dfb,ff12,Point (906980.7207283 6550304.97315954),Point (906736.32315476 6550335.52813463),Point (906595.72670333 6550598.70621165),Point (907032.95563938 6550121.6369531),NaN


In [69]:
data_part2['CODE_CULTU_2019_y'] = data_part2.apply(lambda x: predict_next_culture_4_years_list(x['CODE_CULTU_2015'],x['CODE_CULTU_2016'],x['CODE_CULTU_2017'],x['CODE_CULTU_2018']), axis=1, result_type='expand')

In [70]:
data_part2['CODE_CULTU_2019_y'].value_counts()

Unknown    1453574
PPH        1067851
VRC         181807
J6S         100032
SNE          85849
            ...   
CHR              2
LDP              2
SAD              2
PAN              2
LIP              2
Name: CODE_CULTU_2019_y, Length: 160, dtype: int64

In [71]:
data_part3 = data_part2[data_part2['CODE_CULTU_2019_y'] == 'Unknown']

In [ ]:
print(data_part1.shape)
print(data_part2.shape)
print(data_part3.shape)

(82551, 26)
(3532253, 26)
(1453574, 26)


##Full CatBoost v1###

In [32]:
X2 = data_part3[[
       'CODE_CULTU_2015', 'CODE_CULTU_2016', 'CODE_CULTU_2017',
       'CODE_CULTU_2018', 'CODE_GROUP_2015',
       'CODE_GROUP_2016', 'CODE_GROUP_2017', 'CODE_GROUP_2018']]


X2 = X2.replace(np.nan, 'unknown', regex=True)

Y2 = data_part3['CODE_CULTU_2019']

In [33]:

cat_features2 = [0, 1, 2, 3, 4, 5, 6, 7]

In [34]:
train_dataset2 = Pool(data=X2,
                     label=Y2,
                     cat_features=cat_features2)

eval_dataset2 = Pool(data=X2,
                    label=Y2,
                    cat_features=cat_features2)

# Initialize CatBoostClassifier
model2 = CatBoostClassifier(iterations=200,
                           #learning_rate=0.25,
                           depth=6,
                           loss_function='MultiClass',  task_type='GPU')

#model = CatBoostClassifier(loss_function='MultiClass',  task_type='GPU')

# Fit model
model2.fit(train_dataset2)  
# Get predicted classes
#preds_class2 = model2.predict(eval_dataset2)
# Get predicted probabilities for each class
#preds_proba2 = model2.predict_proba(eval_dataset2)
# Get predicted RawFormulaVal
#preds_raw2 = model2.predict(eval_dataset2, prediction_type='RawFormulaVal')

Learning rate set to 0.5
0:	learn: 14.3086448	total: 5.54s	remaining: 18m 22s
1:	learn: 1144.3181895	total: 11.1s	remaining: 18m 20s
2:	learn: 3486.0948036	total: 16.7s	remaining: 18m 14s
3:	learn: 3452.6444309	total: 22.2s	remaining: 18m 6s
4:	learn: 3960.3214477	total: 27.7s	remaining: 17m 59s
5:	learn: 3204.8004381	total: 33.2s	remaining: 17m 53s
6:	learn: 2691.2543015	total: 38.7s	remaining: 17m 47s
7:	learn: 2598.3091057	total: 44.2s	remaining: 17m 41s
8:	learn: 2478.0790631	total: 49.7s	remaining: 17m 35s
9:	learn: 2395.2828215	total: 55.3s	remaining: 17m 30s
10:	learn: 2342.2444705	total: 1m	remaining: 17m 24s
11:	learn: 2288.5541321	total: 1m 6s	remaining: 17m 18s
12:	learn: 2270.1778430	total: 1m 11s	remaining: 17m 13s
13:	learn: 2127.8837844	total: 1m 17s	remaining: 17m 7s
14:	learn: 2055.0579138	total: 1m 22s	remaining: 17m 2s
15:	learn: 2073.1800651	total: 1m 28s	remaining: 16m 56s
16:	learn: 2005.3057416	total: 1m 33s	remaining: 16m 51s
17:	learn: 1976.6813441	total: 1m 39

## TEST Predict v1 ##

In [72]:
test2020 = pd.read_csv('/content/drive/MyDrive/Хакатон/testWithCodes2015-2019.csv')

In [ ]:
#Смещаем даты на год, т.к. предсказываем на 2020 по данным 2019-2016, для упрощения пайплана считаем, что предсказываем на 2019 по данным 2018-2015

In [73]:
test2020.rename(columns = {'CODE_CULTU_2015': 'CODE_CULTU_2014', 'CODE_GROUP_2015': 'CODE_GROUP_2014'}, inplace = True)

In [74]:
test2020.rename(columns = {'CODE_CULTU_2016': 'CODE_CULTU_2015', 'CODE_GROUP_2016': 'CODE_GROUP_2015'}, inplace = True)

In [75]:
test2020.rename(columns = {'CODE_CULTU_2017': 'CODE_CULTU_2016', 'CODE_GROUP_2017': 'CODE_GROUP_2016'}, inplace = True)

In [76]:
test2020.rename(columns = {'CODE_CULTU_2018': 'CODE_CULTU_2017', 'CODE_GROUP_2018': 'CODE_GROUP_2017'}, inplace = True)

In [77]:
test2020.rename(columns = {'CODE_CULTU_2019': 'CODE_CULTU_2018', 'CODE_GROUP_2019': 'CODE_GROUP_2018'}, inplace = True)

In [78]:
gm_test_pred = pd.merge(gm_uniq, test2020,  how='left', left_on=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'], right_on = ['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'])


In [79]:
gm_test_pred = gm_pred.reindex(columns=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018','CODE_CULTU_2019_y'])

In [80]:
test_pred = pd.merge(test2020, gm_pred,  how='left', left_on=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'], right_on = ['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'])

In [81]:
test_pred['CODE_CULTU_2019_y'].value_counts()

GFP    1
Name: CODE_CULTU_2019_y, dtype: int64

In [82]:
#nms[nms.name.notnull()]
test_part1 = test_pred[data_pred.CODE_CULTU_2019_y.notnull()].copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [46]:
test_part1.head(2)

,Unnamed: 0,CODE_CULTU_2018,CODE_GROUP_2018,centroid,CODE_CULTU_2017,CODE_GROUP_2017,CODE_CULTU_2016,CODE_GROUP_2016,CODE_CULTU_2015,CODE_GROUP_2015,CODE_CULTU_2014,CODE_GROUP_2014,cultuCode,groupCode,CODE_CULTU_2019_y


In [83]:
test_part2 = test_pred[test_pred.CODE_CULTU_2019_y.isna()].copy()

In [84]:
test_part2['CODE_CULTU_2019_y'] = test_part2.apply(lambda x: predict_next_culture_4_years_list(x['CODE_CULTU_2015'],x['CODE_CULTU_2016'],x['CODE_CULTU_2017'],x['CODE_CULTU_2018']), axis=1, result_type='expand')

In [85]:
test_part2['CODE_CULTU_2019_y'].value_counts()

Unknown    13
BTA         9
PPH         1
FAG         1
GFP         1
J6S         1
Name: CODE_CULTU_2019_y, dtype: int64

In [86]:
test_part3 = test_part2[test_part2['CODE_CULTU_2019_y'] == 'Unknown']

In [87]:
test_part3.head(2)

,Unnamed: 0,CODE_CULTU_2018,CODE_GROUP_2018,centroid,CODE_CULTU_2017,CODE_GROUP_2017,CODE_CULTU_2016,CODE_GROUP_2016,CODE_CULTU_2015,CODE_GROUP_2015,CODE_CULTU_2014,CODE_GROUP_2014,cultuCode,groupCode,CODE_CULTU_2019_y
0,0,BTH,1,Point (625771.91344838 7090455.27713006),PTC,25,BTH,1,BTN,24,BTH,1,abaca,abaca,Unknown
1,1,ORP,3,Point (628059.94205405 7090942.4387415),BTH,1,CZH,5,BTH,1,ORP,3,abcba,abcba,Unknown


In [88]:
X_test = test_part3[[
       'CODE_CULTU_2015', 'CODE_CULTU_2016', 'CODE_CULTU_2017',
       'CODE_CULTU_2018', 'CODE_GROUP_2015',
       'CODE_GROUP_2016', 'CODE_GROUP_2017', 'CODE_GROUP_2018']]

X_test = X_test.replace(np.nan, 'unknown', regex=True)

In [89]:
cat_features_test = [0, 1, 2, 3, 4, 5, 6, 7]

In [90]:
test_dataset2 = Pool(data=X_test,
                    cat_features=cat_features_test)


# Get predicted classes
preds_class2 = model2.predict(test_dataset2)

In [91]:
X_test.shape

(13, 8)

In [92]:
test_part3['CODE_CULTU_2019_y'] = preds_class2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [93]:
test_part2_2 = test_part2[test_part2['CODE_CULTU_2019_y'] != 'Unknown']

final_data = pd.concat([test_part1,test_part2_2,test_part3])

In [ ]:
###Корректировка явных ошибок CatBoost### (Предсказание значения, которое не встречалось в выборке, заменяем на последнее значение в выборке)

In [94]:
test_part3_1 = test_part3.loc[(test_part3['CODE_CULTU_2019_y'] != test_part3['CODE_CULTU_2015']) & (test_part3['CODE_CULTU_2019_y'] != test_part3['CODE_CULTU_2016']) & (test_part3['CODE_CULTU_2019_y'] != test_part3['CODE_CULTU_2017']) & (test_part3['CODE_CULTU_2019_y'] != test_part3['CODE_CULTU_2018']) ]

In [95]:
test_part3_2 = test_part3.loc[(test_part3['CODE_CULTU_2019_y'] == test_part3['CODE_CULTU_2015']) | (test_part3['CODE_CULTU_2019_y'] == test_part3['CODE_CULTU_2016']) | (test_part3['CODE_CULTU_2019_y'] == test_part3['CODE_CULTU_2017']) | (test_part3['CODE_CULTU_2019_y'] == test_part3['CODE_CULTU_2018']) ]

In [96]:
test_part3_1[['CODE_CULTU_2019_y','CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018']]

,CODE_CULTU_2019_y,CODE_CULTU_2015,CODE_CULTU_2016,CODE_CULTU_2017,CODE_CULTU_2018
3,J5M,BTH,CZH,FAG,BTH
12,TRN,AVP,CZH,BTH,PTC
14,PTR,BTH,BTH,BTH,LIF


In [97]:
test_part3_1['CODE_CULTU_2019_y'] = test_part3_1['CODE_CULTU_2018']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [98]:
final_data_test = pd.concat([test_part1,test_part2_2,test_part3_1,test_part3_2])

In [99]:
test2020.rename(columns = {'CODE_CULTU_2019_y': 'CODE_CULTU_2020'}, inplace = True)

In [ ]:
pd.DataFrame(final_data_test[['centroid','CODE_CULTU_2020']]).to_csv("/content/drive/MyDrive/Хакатон/predict2020.csv",index=False,header=True)

In [ ]:
print(test_part1.shape)
print(test_part2_2.shape)
print(test_part3.shape)

print(test_part3_1.shape)
print(test_part3_2.shape)

(4554, 19)
(230542, 19)
(166549, 20)
(31520, 20)
(135029, 20)


##Full CatBoost v2###

In [103]:
X2 = data_part3[['RotationLength', 'RotationCount',
       'CODE_CULTU_2015', 'CODE_CULTU_2016', 'CODE_CULTU_2017',
       'CODE_CULTU_2018', 'CODE_GROUP_2015',
       'CODE_GROUP_2016', 'CODE_GROUP_2017', 'CODE_GROUP_2018',
       'hzs', 'kg_id', 'ff_id']]


X2 = X2.replace(np.nan, 'unknown', regex=True)

Y2 = data_part3['CODE_CULTU_2019']

In [104]:
cat_features2 = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [105]:
train_dataset2 = Pool(data=X2,
                     label=Y2,
                     cat_features=cat_features2)

eval_dataset2 = Pool(data=X2,
                    label=Y2,
                    cat_features=cat_features2)

# Initialize CatBoostClassifier
model2 = CatBoostClassifier(iterations=200,
                           #learning_rate=0.25,
                           depth=6,
                           loss_function='MultiClass',  task_type='GPU')

#model = CatBoostClassifier(loss_function='MultiClass',  task_type='GPU')

# Fit model
model2.fit(train_dataset2)  
# Get predicted classes
#preds_class2 = model2.predict(eval_dataset2)
# Get predicted probabilities for each class
#preds_proba2 = model2.predict_proba(eval_dataset2)
# Get predicted RawFormulaVal
#preds_raw2 = model2.predict(eval_dataset2, prediction_type='RawFormulaVal')

Learning rate set to 0.5
0:	learn: 14.2548615	total: 5.54s	remaining: 18m 22s
1:	learn: 1157.3926336	total: 11.1s	remaining: 18m 23s
2:	learn: 2730.9945390	total: 16.7s	remaining: 18m 16s
3:	learn: 3467.8993873	total: 22.2s	remaining: 18m 10s
4:	learn: 3646.6503226	total: 27.8s	remaining: 18m 3s
5:	learn: 3703.5423033	total: 33.3s	remaining: 17m 55s
6:	learn: 3297.7398220	total: 38.8s	remaining: 17m 48s
7:	learn: 3194.2178823	total: 44.3s	remaining: 17m 42s
8:	learn: 2844.8642697	total: 49.8s	remaining: 17m 36s
9:	learn: 2565.9293521	total: 55.4s	remaining: 17m 31s
10:	learn: 2355.9967914	total: 1m	remaining: 17m 26s
11:	learn: 2229.3086915	total: 1m 6s	remaining: 17m 20s
12:	learn: 2053.4022320	total: 1m 11s	remaining: 17m 15s
13:	learn: 1989.3632220	total: 1m 17s	remaining: 17m 10s
14:	learn: 1918.1495321	total: 1m 23s	remaining: 17m 4s
15:	learn: 1868.2893998	total: 1m 28s	remaining: 16m 59s
16:	learn: 1811.7412007	total: 1m 34s	remaining: 16m 53s
17:	learn: 1784.1451539	total: 1m 3

## TEST Predict v2 ##

In [126]:
test2020 = pd.read_csv('/content/drive/MyDrive/Хакатон/testdataset2015-2019_full.csv')

In [ ]:
#Смещаем даты на год, т.к. предсказываем на 2020 по данным 2019-2016, для упрощения пайплана считаем, что предсказываем на 2019 по данным 2018-2015

In [127]:
test2020.rename(columns = {'CODE_CULTU_2015': 'CODE_CULTU_2014', 'CODE_GROUP_2015': 'CODE_GROUP_2014'}, inplace = True)

In [128]:
test2020.rename(columns = {'CODE_CULTU_2016': 'CODE_CULTU_2015', 'CODE_GROUP_2016': 'CODE_GROUP_2015'}, inplace = True)

In [129]:
test2020.rename(columns = {'CODE_CULTU_2017': 'CODE_CULTU_2016', 'CODE_GROUP_2017': 'CODE_GROUP_2016'}, inplace = True)

In [130]:
test2020.rename(columns = {'CODE_CULTU_2018': 'CODE_CULTU_2017', 'CODE_GROUP_2018': 'CODE_GROUP_2017'}, inplace = True)

In [131]:
test2020.rename(columns = {'CODE_CULTU_2019': 'CODE_CULTU_2018', 'CODE_GROUP_2019': 'CODE_GROUP_2018'}, inplace = True)

In [132]:
gm_test_pred = pd.merge(gm_uniq, test2020,  how='left', left_on=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'], right_on = ['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'])


In [133]:
gm_test_pred = gm_pred.reindex(columns=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018','CODE_CULTU_2019_y'])

In [134]:
test_pred = pd.merge(test2020, gm_pred,  how='left', left_on=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'], right_on = ['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'])

In [135]:
test_pred['CODE_CULTU_2019_y'].value_counts()

GFP    1
Name: CODE_CULTU_2019_y, dtype: int64

In [136]:
test_part1 = test_pred[data_pred.CODE_CULTU_2019_y.notnull()].copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [137]:
test_part1.head(2)

,CODE_CULTU_2018,CODE_GROUP_2018,centroid,CODE_CULTU_2017,CODE_GROUP_2017,CODE_CULTU_2016,CODE_GROUP_2016,CODE_CULTU_2015,CODE_GROUP_2015,CODE_CULTU_2014,CODE_GROUP_2014,cultuCode,groupCode,2019,RotationLength,RotationCount,hzs,kg_id,ff_id,CODE_CULTU_2019_y


In [138]:
test_part2 = test_pred[test_pred.CODE_CULTU_2019_y.isna()].copy()

In [139]:
test_part2['CODE_CULTU_2019_y'] = test_part2.apply(lambda x: predict_next_culture_4_years_list(x['CODE_CULTU_2015'],x['CODE_CULTU_2016'],x['CODE_CULTU_2017'],x['CODE_CULTU_2018']), axis=1, result_type='expand')

In [140]:
test_part2['CODE_CULTU_2019_y'].value_counts()

Unknown    13
BTA         9
PPH         1
FAG         1
GFP         1
J6S         1
Name: CODE_CULTU_2019_y, dtype: int64

In [141]:
test_part3 = test_part2[test_part2['CODE_CULTU_2019_y'] == 'Unknown']

In [142]:
test_part3.head(2)

,CODE_CULTU_2018,CODE_GROUP_2018,centroid,CODE_CULTU_2017,CODE_GROUP_2017,CODE_CULTU_2016,CODE_GROUP_2016,CODE_CULTU_2015,CODE_GROUP_2015,CODE_CULTU_2014,CODE_GROUP_2014,cultuCode,groupCode,2019,RotationLength,RotationCount,hzs,kg_id,ff_id,CODE_CULTU_2019_y
0,BTH,1,Point (625771.91344838 7090455.27713006),PTC,25,BTH,1,BTN,24,BTH,1,abaca,abaca,BTH,3,1,9a,Cfb,ff16,Unknown
1,ORP,3,Point (628059.94205405 7090942.4387415),BTH,1,CZH,5,BTH,1,ORP,3,abcba,abcba,ORP,3,1,9a,Cfb,ff16,Unknown


In [143]:
X_test = test_part3[['RotationLength', 'RotationCount',
       'CODE_CULTU_2015', 'CODE_CULTU_2016', 'CODE_CULTU_2017',
       'CODE_CULTU_2018', 'CODE_GROUP_2015',
       'CODE_GROUP_2016', 'CODE_GROUP_2017', 'CODE_GROUP_2018',
       'hzs', 'kg_id', 'ff_id']]

X_test = X_test.replace(np.nan, 'unknown', regex=True)

In [144]:
cat_features_test = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [145]:
test_dataset2 = Pool(data=X_test,
                    cat_features=cat_features_test)


# Get predicted classes
preds_class2 = model2.predict(test_dataset2)

In [146]:
X_test.shape

(13, 13)

In [147]:
test_part3['CODE_CULTU_2019_y'] = preds_class2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [148]:
test_part2_2 = test_part2[test_part2['CODE_CULTU_2019_y'] != 'Unknown']

In [ ]:
###Корректировка явных ошибок CatBoost### (Предсказание значения, которое не встречалось в выборке, заменяем на последнее значение в выборке)

In [149]:
test_part3_1 = test_part3.loc[(test_part3['CODE_CULTU_2019_y'] != test_part3['CODE_CULTU_2015']) & (test_part3['CODE_CULTU_2019_y'] != test_part3['CODE_CULTU_2016']) & (test_part3['CODE_CULTU_2019_y'] != test_part3['CODE_CULTU_2017']) & (test_part3['CODE_CULTU_2019_y'] != test_part3['CODE_CULTU_2018']) ]

In [150]:
test_part3_2 = test_part3.loc[(test_part3['CODE_CULTU_2019_y'] == test_part3['CODE_CULTU_2015']) | (test_part3['CODE_CULTU_2019_y'] == test_part3['CODE_CULTU_2016']) | (test_part3['CODE_CULTU_2019_y'] == test_part3['CODE_CULTU_2017']) | (test_part3['CODE_CULTU_2019_y'] == test_part3['CODE_CULTU_2018']) ]

In [151]:
test_part3_1[['CODE_CULTU_2019_y','CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018']]

,CODE_CULTU_2019_y,CODE_CULTU_2015,CODE_CULTU_2016,CODE_CULTU_2017,CODE_CULTU_2018
1,TRN,BTH,CZH,BTH,ORP
3,MIE,BTH,CZH,FAG,BTH
13,PCH,BTH,CZH,BTH,LIF
19,CHU,BTH,BTN,BTH,LIF


In [152]:
test_part3_1['CODE_CULTU_2019_y'] = test_part3_1['CODE_CULTU_2018']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [153]:
final_data_test = pd.concat([test_part1,test_part2_2,test_part3_1,test_part3_2])

In [154]:
test2020.rename(columns = {'CODE_CULTU_2019_y': 'CODE_CULTU_2020'}, inplace = True)

In [ ]:
pd.DataFrame(final_data_test[['centroid','CODE_CULTU_2020']]).to_csv("/content/drive/MyDrive/Хакатон/predict2020_v2.csv",index=False,header=True)